#Code documentation
- class World: Initializes the data and creates the 100 percentiles
    - rate_ret: Already 100 percentiles (fit the weird exp curve)
    - income/wealth array: Already 100 percentiles (from data)
    - sav_rate: ["90","10","1"]
    - cap_gains: ["90","10","1"]

In [9]:
# world.py
"""
Created on Wed Jun 27 19:08:17 2018
@author: ymamo
Creates the policy environment
"""

import pickle
class World:
    #Initiates variables for master equation
    def __init__(self, model): 
        self.model = model
        self.rate_ret = self.pickle_open("data_abm/year2rate_return.dump")
        #print (self.rate_ret.keys())
        #stop
        self.sav_rate = self.pickle_open("data_abm/year2saving.dump")
        self.cap_gains = self.pickle_open("data_abm/year2capital_gains.dump")
        
        self.sav_rate = self.create_percentiles(self.sav_rate,perc=[90,9,1])
        self.cap_gains = self.create_percentiles(self.cap_gains,perc=[90,9,1])
        
        #self.growth_rate = 1.1
        self.income_array = self.pickle_open("data_abm/year2income.dump")
        self.wealth_array = self.pickle_open("data_abm/year2wealth.dump")
        
    #Reads in data to update varibales for master equation     
    def create_percentiles(self,values,perc):
        new_values = []
        #for each of the groups
        for p in perc:
            #use that value for all the percentiles
            val = values.pop(0)
            for i in range(p):
                new_values.append(val)
        
        return new_values
    
    def pickle_open(self, file):         
        with open(file, 'rb') as f:
            data = pickle.load(f)
            
        return data        

In [ ]:
# decileagent.py
"""
Created on Thu Jun 21 18:22:09 2018

@author: ymamo
"""

from mesa import Agent
import random 

class DecileAgent(Agent):
    # Initiaite Each Agent
    def __init__(self, unique_id, model, wealth, rank):
        super().__init__(unique_id, model)
        # initial wealth
        self.wealth = wealth
        # which decile (economic class) agent belongs too
        self.rank = rank
        
    ############################################################################
    #   AGENT FUNCTIONS WHICH OCCUR EACH STEP
    ##############################################################################
    
    def update(self):
        decile = self.rank//100
    

        rate_return = self.model.world.rate_ret[self.model.time][decile]
        income_yr = self.model.world.income_array[self.model.time][decile]
        cap_gains = self.model.world.cap_gains[self.model.time][decile]
        sav_rate = self.model.world.sav_rate[self.model.time][decile]
        
        self.wealth = (1+cap_gains) * (self.wealth + sav_rate*(income_yr + self.wealth*rate_return))
       
        '''
        if random.random() < 0.5 : 
           self.income = 1 + self.income
        else: 
           self.income = change_income + 1 + self.income
        '''   
    
    def step(self):
                
        self.update()
        
        
    
        

In [ ]:
# data-inequality
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 21 18:15:05 2018

@author: ymamo
"""

from mesa import Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


import numpy as np
import pandas as pd


class InequalityModel(Model):
    
    '''
    Model to creat 
    
    '''


    
    def __init__(self):
        ###################################
        #   SET INITIAL NUMBERS
        ###################################
        self.num_agents = 10000
        self.wealth_per = np.loadtxt("data_abm/init_wealth_percentile.txt")
        self.num_decs = int(self.num_agents/100)
        self.decile = self.create_deciles(self.wealth_per)
        ###################################
        #   Create Income and Top Down World
        ######################################
        self.schedule = RandomActivation(self)
        self.rank_list = []
        self.world = World(self)
        self.time = 1966
        #####################################
        # Create agent and add to schedule
        ########################################
        for i in range(self.num_agents):
            #get correct decile for agent population
            pos = i//100
            #print (len(self.decile[pos]))
            wealth=self.decile[pos][i%self.num_decs]
            a = DecileAgent(i, self, wealth,i)
            self.schedule.add(a)
            self.rank_list.append(a)
        self.datacollector = DataCollector(agent_reporters = {"Wealth" : "wealth", "Rank":"rank"})
        
    
    ##################################################################
    # HELPER FUNCTION FOR INIT
    #################################################################         
    def create_deciles(self, wealth):
        wealth_map = wealth
        #account for upper wealth threshold
        wealth_map = np.append(wealth_map, 100000000000.0)
        deciles = []
        income_dics = []
        for i in range(100):
            array_w = list(np.random.uniform(wealth_map[i],wealth_map[i+1], self.num_decs))
            array_w.sort()
            deciles.append(array_w)
       
        #print (len(deciles[0]))
        return deciles
    
    def rerank(self):
        self.rank_list.sort(key=lambda x: x.wealth)
        for a in self.rank_list:
            a.rank = self.rank_list.index(a)
    ###########################################################################
    #  STEP FUNCTION --- WHAT OCCURS EACH YEAR
    ###########################################################################
    
    def step(self):
        self.rerank()
        self.schedule.step()
        self.time += 1
        self.datacollector.collect(self)
        

###############################################################################
#
#
#                         RUN THE MODEL
#
################################################################################
# Create an instance of the model
test = InequalityModel()

#Range indicate number of steps
for i in range(46):
    print(test.time)
    test.step()
    
results = test.datacollector.get_agent_vars_dataframe()
results.to_csv("results.csv")

